In [2]:
# Importamos las dependencias necesarias
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# NLTK
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
# Lectura del csv
stop_words_en = stopwords.words('english')
stop_words_en.extend(["€", ",", ".", ";", ":", "%", "'", "=", "[", "]", "/", "<", ".-", "e.g", "i.e", "~", "--", "without", "#"])
stop_words_en.extend(["``", "(", ")", "this", "the", "that", "?", "!", "≤", "™", ">", "≥/", "≥", "-", "$" , "’", "-", "  ", "&"])
stop_words_en.extend(["would", "...", "us", 'i\'m', "gonna", "wanna", "imma"])

df = pd.read_csv('The_Reserve_PPR.csv')
df.head(2)
#print(df['Review Text'].isnull().sum()) # comprobar que no hay filas vacías de la columna. En este caso es 9
df.dropna(subset=['Review Text'], axis=0, inplace=True)

# Tokenizamos y eliminamos las stop words
df["Review Text"] = df["Review Text"].astype("str") # Convertimos la columna en una string/object
# df["Review Text"].dtypes # dtype('O')

In [4]:
# Lematiza las columnas del un texto
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def tokenizar(texto_raw):
    tokens = nltk.tokenize.word_tokenize(texto_raw)
    return tokens

wnl = nltk.stem.WordNetLemmatizer()
def lemmatize(s):
    s = [wnl.lemmatize(word) for word in s]
    return s

# Tokeniza las filas con el texto
df["Review Text"] = df.apply(lambda row: tokenizar(row["Review Text"].lower()), axis=1)

## Elimina stopwords y tokeniza en una nueva columna
df["Review Text"] = df["Review Text"].apply(lambda x: ' '.join([word for word in x if word not in (stop_words_en)]))

# Tokeniza las filas con el texto y las guarda en una nueva columna
df["Review Text"] = df.apply(lambda row: tokenizar(row["Review Text"]), axis=1)
print(df["Review Text"])

#df["Review Text"]
df["lemma"] = df["Review Text"].apply(lambda x: lemmatize(x))

0       [elegant, high, class, minute, walk, sure, fee...
1       [awesome, time, resort, well, maintained, staf...
2       [thanks, much, jesus, avalino, willy, pena, re...
3       [mosquitoes, like, please, please, spray, ever...
4       [concierge, zenovia, pleasure, work, kept, inf...
                              ...                        
1088    [start, finish, relaxing, vacation, food, good...
1089    [hotel, amazing, exceeded, expectations, upgra...
1090    [great, experience, staff, friendly, alejandro...
1091    [stayed, reserve, concierge, area, reserved, h...
1092    [wonderful, family, vacation, kids, club, staf...
Name: Review Text, Length: 1092, dtype: object


In [6]:
## Sacar TOPIC MODELING
# Inicializa el modelo "en", solo con el tagger, deshabilitando el parser y el ner (eficiencia)
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

# Lematización con solo sustantivos, adjetivos y adverbios.
# Nos servirá posteriormente como el corpus para el topic modeling
data_lemmatized = lemmatization(df['lemma'], allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#print(data_lemmatized)

[['elegant', 'high', 'class', 'minute', 'walk', 'sure', 'feel', 'big', 'clean', 'modern', 'treat', 'kindly', 'tell', 'customer', 'satisfaction', 'main', 'priority', 'even', 'know', 'employee', 'heart', 'arrive', 'greet', 'smile', 'help', 'luggage', 'let', 'talk', 'helpful', 'whenever', 'need', 'mostly', 'waitress-', 'always', 'smile', 'attentive', 'bartender', 'also', 'attentive', 'communicative', 'm', 'gon', 'miss', 'people', 'definately', 'come', 'back', 'resort'], ['awesome', 'time', 'resort', 'well', 'maintain', 'staff', 'friendly', 'welcome', 'make', 'sure', 'great', 'time', 'family', 'concierge', 'milly', 'good', 'get', 'reservation', 'help', 'need', 'kid', 'wonderful', 'time', 'great', 'vacation', 'thing', 'timeshare', 'wish', 'spend', 'time', 'splendid', 'feel'], ['thank', 'much', 'team', 'constant', 'great', 'service', 'fun', 'attitude', 'drink', 'wonderful', 'amazing', 'always', 'make', 'even', 'well', 'thank', 'make', 'medium', 'proud', 'attentive', 'team', 'member', 'experi

In [7]:
# Crea el Dictionario y el Corpus necesario para el Topic Modeling

# Crea el Dictionario con los tokens lematizados de las llamadas buenas
id2word = corpora.Dictionary(data_lemmatized)

# Crea el Corpus
texts = data_lemmatized

# Term Frequency en el documento
corpus = [id2word.doc2bow(text) for text in texts]

# Contruimos el modelo LDA
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, # Establece el número de topics por texto
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10, # alpha and eta are hiperparámetros que afectan a la escasez de topics.                                           alpha='auto',
                                           per_word_topics=True)

# Imprime 10 keywords de los topics que se impriman en pantalla
#pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

# Calcular el modelo de perplejidad y puntuación de coherencia
# Esto nos ayudará a calcular lo bueno o malo que es un modelo de topics

In [8]:
# Calcular el modelo de perplejidad y puntuación de coherencia
# Esto nos ayudará a calcular lo bueno o malo que es un modelo de topics

# Modelo de Perplejidad
# Distribución de probabilidad sobre un conjunto de textos
#print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# -7.103914580268614

# Puntuación de Coherencia
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda)
# Con 5 topics por texto, la coherencia es de 0.46522933417849427

In [9]:
# Imprime 10 keywords de 2 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Interpretación del modelo:

#  Topic 0 se representa como '0.003*"día" + 0.003*"valer" + 0.003*"préstamo" + 0.003*"fecho" + ...
# Significa que los 10 keywords que contribuyen a ese topic son "día", "valer", "préstamo", "contar", "fecho", "tarjeta"...
# y qe el peso de "día" es de 0.003. 
# Los pesos reflejan lo importante que es esa keyword para ese topic.

[(0,
  '0.043*"general" + 0.036*"hospitality" + 0.036*"seafood" + 0.028*"recently" '
  '+ 0.026*"normally" + 0.024*"select" + 0.022*"feature" + 0.021*"cleanliness" '
  '+ 0.021*"casino" + 0.021*"bug"'),
 (1,
  '0.088*"memorable" + 0.053*"anniversary" + 0.047*"celebrate" + '
  '0.045*"single" + 0.020*"flight" + 0.017*"arrangement" + 0.017*"flower" + '
  '0.017*"beat" + 0.014*"opportunity" + 0.013*"rude"'),
 (2,
  '0.102*"mean" + 0.057*"nightly" + 0.010*"up" + 0.005*"exact" + 0.001*"frig" '
  '+ 0.000*"destination" + 0.000*"efficient" + 0.000*"girlfriend" + '
  '0.000*"somewhat" + 0.000*"suit"'),
 (3,
  '0.031*"amaze" + 0.000*"exceed" + 0.000*"overwhelmed" + 0.000*"dip" + '
  '0.000*"recommendation" + 0.000*"match" + 0.000*"housekeeper" + '
  '0.000*"tremendous" + 0.000*"lie" + 0.000*"assortment"'),
 (4,
  '0.040*"make" + 0.022*"family" + 0.021*"great" + 0.019*"time" + 0.018*"stay" '
  '+ 0.016*"always" + 0.016*"go" + 0.016*"resort" + 0.015*"thank" + '
  '0.015*"vacation"'),
 (5,
  '0.03

In [10]:
# Vamos a visualizar los topic-keywords
# Una vez construido el modelo LDA, vamos a examinar los topics y sus respectivas keywords. 
# Para ello vamos a usar la librería pyLDAvis

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

# Cada burbuja representa un topic. Cuanto más grande la burbuja, más importante.
# Un buen modelo de topics tendrá burbujas grandes que no se solaparán entre ellas.
# Por el contrario, uno malo tendrá muchas burbujitas solapadas en una región del chart. 
# Si pasas el cursor por encima de una burbuja (el topic), verás que el chart se refresca. 
# Las palabras de la derecha son las keywords.

C:\Users\Usuario\Anaconda3\envs\study_env\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13     0.345151  0.005199       1        1  44.923779
4      0.388415 -0.058802       2        1  28.766207
5      0.277672  0.157652       3        1  14.371264
11     0.242286 -0.232620       4        1   4.083878
6      0.076309  0.246609       5        1   3.626994
7     -0.087637 -0.036093       6        1   1.400087
8     -0.146379 -0.018617       7        1   0.626049
1     -0.144168 -0.018543       8        1   0.625767
0     -0.146063 -0.016276       9        1   0.481141
14    -0.141544 -0.006218      10        1   0.319376
9     -0.137003 -0.005908      11        1   0.215935
2     -0.137725 -0.007196      12        1   0.204185
10    -0.129480 -0.002924      13        1   0.121048
12    -0.129623 -0.002992      14        1   0.117278
3     -0.130212 -0.003271      15        1   0.117006, topic_info=     Category        Freq           Term       Total  loglift  logprob
52    Default  827.000000          great  827.000000  30.0000  30.0000
60    Default  722.000000          staff  722.000000  29.0000  29.0000
51    Default  841.000000           good  841.000000  28.0000  28.0000
80    Default  780.000000        service  780.000000  27.0000  27.0000
193   Default  549.000000        reserve  549.000000  26.0000  26.0000
...       ...         ...            ...         ...      ...      ...
1771  Topic15    0.017556         though    0.718387   3.0391  -8.4731
2464  Topic15    0.017556           blow    0.717245   3.0407  -8.4731
1592  Topic15    0.017556        website    0.717371   3.0405  -8.4731
1555  Topic15    0.017556       language    0.719847   3.0371  -8.4731
2468  Topic15    0.017556  disappointing    0.718744   3.0386  -8.4731

[836 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
906       8  0.800780         10th
822       8  0.647228         24/7
1466      6  0.818183          5th
952       7  0.961342     absolute
103       2  0.952215  accommodate
...     ...       ...          ...
178       1  0.415598        young
178       3  0.120657        young
178       5  0.455817        young
1319      6  0.941928        yummy
1170      4  0.950968        zumba

[729 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 5, 6, 12, 7, 8, 9, 2, 1, 15, 10, 3, 11, 13, 4])